In [1]:
import numpy as np
from scipy import interpolate
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.io

import nelpy as nel  # recommended import for nelpy
import nelpy.plotting as npl  # recommended import for the nelpy plotting library

%matplotlib inline

from nelpy import EpochArray, AnalogSignalArray, SpikeTrainArray, BinnedSpikeTrainArray

In [2]:
nel

<module 'nelpy' from '/home/shayok/Documents/Code/nelpy/nelpy/__init__.py'>

# TODO
1. Look at SpikeTrainArray<br>
2. Implement remaining "Stuff to add" <br>
3. Scrutinize ASA in terms of efficiency and check for obvious improvements<br>
4. Push changes!<br>
5. Test out filtering stuff<br>
6. Add in ripple detection functions <br>
7. Add in Trodes data extraction <br>

# Stuff to add
- support single values ASA([1]) -- DONE!<br>
- empty values ASA([]) -- FIXED!<br>
- out of bounds errors! -- FIXED!<br>
- Subset functionality https://github.com/eackermann/nelpy/issues/126<br>
- mean, std, min, max, with axis flag https://github.com/eackermann/nelpy/issues/127 -- DONE!<br>
- empty object pattern and attributes....https://github.com/eackermann/nelpy/issues/85

# Bugs found
- Time for multidimensional data without timestamps or time stamps in seconds provided may not have been calculated correctly -- FIXED <br><br>

# Basic AnalogSignalArray Functionality

## Just tryin to break it a lil bit

In [3]:
my_lfp = AnalogSignalArray([])
# my_lfp.ydata
# my_lfp.support
my_lfp

<empty AnalogSignal at 0x7f81b1184c88>

In [4]:
asa = AnalogSignalArray(['a'])
asa

TypeError: Unsupported type! integer or floating point expected

In [5]:
asa = AnalogSignalArray([1,2,3])
asa.support.duration

/home/shayok/Documents/Code/nelpy/nelpy/objects.py:1472: UserWarning: support created with given ydata! support is entire signal


2.0

In [6]:
asa = AnalogSignalArray([[1],[2],[3]])
print(asa.mean())
print(asa.std())
print(asa.min())
print(asa.max())
asa

[ 1.  2.  3.]
[ 0.  0.  0.]
[ 1.  2.  3.]
[ 1.  2.  3.]


/home/shayok/Documents/Code/nelpy/nelpy/objects.py:1472: UserWarning: support created with given ydata! support is entire signal


<AnalogSignalArray at 0x7f817798f470: 3 signals> for a total of 0 milliseconds

In [7]:
asa2 = AnalogSignalArray([[1,2],[2,2],[3,2]])
asa2[EpochArray([2,3])]

/home/shayok/Documents/Code/nelpy/nelpy/objects.py:1472: UserWarning: support created with given ydata! support is entire signal
/home/shayok/Documents/Code/nelpy/nelpy/objects.py:65: UserWarning: No sampling frequency has been specified!
/home/shayok/Documents/Code/nelpy/nelpy/objects.py:65: UserWarning: No sampling frequency has been specified!
/home/shayok/Documents/Code/nelpy/nelpy/objects.py:824: UserWarning: meta data is not available
/home/shayok/Documents/Code/nelpy/nelpy/objects.py:65: UserWarning: No sampling frequency has been specified!
/home/shayok/Documents/Code/nelpy/nelpy/objects.py:824: UserWarning: meta data is not available
/home/shayok/Documents/Code/nelpy/nelpy/objects.py:65: UserWarning: No sampling frequency has been specified!
/home/shayok/Documents/Code/nelpy/nelpy/objects.py:1027: UserWarning: sampling rates are different; intersecting along time only and throwing away fs
/home/shayok/Documents/Code/nelpy/nelpy/objects.py:1493: UserWarning: Support specified i

<empty AnalogSignal at 0x7f817798f240>

In [21]:
cow = np.array([[1,3],[2,3]])
np.zeros([0]).shape

(0,)

In [14]:
cow = nel.SpikeTrainArray([[1,2,3],[4.5]], support=EpochArray(empty=True))
cow.time


array([], shape=(2, 0), dtype=float64)

In [9]:
asa = nel.AnalogSignalArray([[3,4,5],[5,6,7]], tdata=[2,3,4], support=nel.EpochArray(empty=True))
print(asa.n_signals)

2


/home/shayok/Documents/Code/nelpy/nelpy/objects.py:1493: UserWarning: Support specified is empty
/home/shayok/Documents/Code/nelpy/nelpy/objects.py:1435: UserWarning: support created with specified epoch array but no specified sampling rate
/home/shayok/Documents/Code/nelpy/nelpy/objects.py:1437: UserWarning: Support is empty. Empty AnalogSignalArray returned


## Ting Data

In [ ]:
tingData = scipy.io.loadmat("/home/shayok/Downloads/simiplified(Newton, 2015-03-11_15-09-22).mat")
eeg = tingData["eeg"]
eegTimeStamps = tingData["eegtimestamp"]

In [ ]:
plt.plot(eegTimeStamps, eeg)
plt.show()

In [ ]:
my_time_series = AnalogSignalArray([eeg],tdata=eegTimeStamps,)
npl.plot(my_time_series)
plt.show()
plt.plot(eegTimeStamps, eeg)
my_time_series

In [ ]:
my_time_series.support.durations

In [ ]:
my_time_series.support.merge(gap=0.1)

In [ ]:
my_time_series._support = my_time_series.support.merge(gap=0.1)
my_time_series.support.durations

In [ ]:
my_time_series

## My Data

In [ ]:
myFile = np.load('/home/shayok/Downloads/LFP.npz')
lfp = myFile['dataT2C1']*0.195
onlineRippleDetect = myFile['onlineRippleDetect']
offlineRippleDetect = myFile['offlineRippleDetections']
timeStampsSeconds = myFile['timeStampsSeconds']
timeStamps = myFile['timeStamps']
plt.plot(timeStampsSeconds,lfp)
plt.show()

In [ ]:
my_lfp = AnalogSignalArray([lfp],tdata=timeStamps,fs=30000,step=10)
print("Mean")
print(my_lfp.mean())
print(np.mean(np.transpose([lfp,onlineRippleDetect]),axis=0))
print("Standard Deviation")
print(my_lfp.std())
print(np.std(np.transpose([lfp,onlineRippleDetect]),axis=0))
print("min")
print(my_lfp.min())
print(np.amin(np.transpose([lfp,onlineRippleDetect]),axis=0))
print("max")
print(my_lfp.max())
print(np.amax(np.transpose([lfp,onlineRippleDetect]),axis=0))

In [ ]:
npl.plot(my_lfp)

In [ ]:
etiennes_lfp=AnalogSignalArray([[1,5],[2,5,],[3,5]])
etiennes_lfp.min()

In [ ]:
my_lfp.n_signals

In [ ]:
partialSupport = EpochArray([[1200,1250], [1270,1290], [1345,1370],[1380,1399]],fs=1)
my_lfp = AnalogSignalArray([lfp,onlineRippleDetect,offlineRippleDetect],tdata=timeStamps,support=partialSupport,fs=30000,step=10)
npl.plot(my_lfp)
print(my_lfp.support)
my_lfp

In [ ]:
my_lfp.support.durations

In [ ]:
my_lfp.n_epochs

In [ ]:
npl.plot(my_lfp._subset([1,2]))

In [ ]:
clipped_lfp = my_lfp.clip(-1000,1000)
npl.plot(clipped_lfp)
means = [snippet.mean() for snippet in clipped_lfp]
print(means)
print(clipped_lfp.support)
print(clipped_lfp)
npl.plot(clipped_lfp.support, means)

In [ ]:
partialSupportForEtienne = EpochArray([[1200,1200.5], [1201.2,1201.3], [1201.5,1201.8],
                                       [1202,1202.2], [1202.5,1203]],fs=1)
mySuperRestrictedLFP = clipped_lfp[partialSupportForEtienne]
npl.plot(mySuperRestrictedLFP)

In [ ]:
print(mySuperRestrictedLFP.n_signals, mySuperRestrictedLFP.n_epochs)
print(mySuperRestrictedLFP.ydata.shape)
print(mySuperRestrictedLFP.mean(), np.mean(mySuperRestrictedLFP.ydata,axis=0))
print(np.mean(mySuperRestrictedLFP.ydata,axis=1).shape)

In [ ]:
mynewlfp = clipped_lfp[2]
npl.plot(mynewlfp)

# Random

In [ ]:
cow = np.array([[1,3,3],[3,1,3],[3,1,3],[3,1,2]])
cow.shape

In [ ]:
cow[:,[0,1]]

In [ ]:
cmoo = "%s cows %s moos" %(4,56)
print(cmoo)

In [ ]:
moo = np.squeeze(cow)
moo.shape

In [ ]:
try:
    if(timeStamps.shape[1]==cow.shape[1]):
        print("moo")
except:
    print("heyy")

# Interpolation

In [ ]:
print(timeStamps[0])
print(timeStamps[1])

In [ ]:
cow1 = []
cow1.append("moo")

In [ ]:
interpObj = interpolate.interp1d(timeStampsSeconds, lfp)

In [ ]:
xnew = np.arange(timeStampsSeconds[5000],timeStampsSeconds[5010],1/30000)
ynew = interpObj(xnew)
plt.plot(timeStampsSeconds[5000:5010], lfp[5000:5010],'o',xnew,ynew,'-')
plt.show()

In [ ]:
print(my_lfp.ydata[5000],lfp[5000])

In [ ]:
my_lfp = AnalogSignalArray([lfp],tdata=timeStampsSeconds,)
my_lfp

In [ ]:
print(np.min(xnew),np.max(xnew))

In [ ]:
print(timeStampsSeconds[1993])
print(my_lfp.time[3000])
print(my_lfp.time[4000])

In [ ]:
print(my_lfp.tdata[0]/30000,timeStamps[0]/30000,timeStampsSeconds[0])

In [ ]:
ytestnew = my_lfp.interp(xnew)
print(xnew.shape)
print((ytestnew).shape)
plt.plot(my_lfp.time[5000:5010], my_lfp.ydata[5000:5010],'o',xnew,np.squeeze(ytestnew),'-')
plt.show()